In [1]:
%load_ext autoreload
%autoreload 2

import os
import dosma as dm
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import pickle
import SimpleITK as sitk

from dosma.scan_sequences import QDess
from dosma import ImageDataFormat

from utils.filter_qmaps import *
from utils.difference_map_tissue import *
from utils.apply_intensity_threshold import *
from utils.apply_volume_threshold import *
from utils.compute_t2 import *
from utils.convert_qdess_dicom2nii import *
from utils.sgd_based_registration import *
from utils.sgd_based_qmap_registration import *
from utils.dosma_segmentation import *
from utils.compute_DSC import *
from utils.compute_intensity_threshold import *
from utils.compute_volume_threshold import * 
from utils.sgd_based_bilateral_qdess_registration import *

import dosma as dm
import os

import numpy as np
import matplotlib.pyplot as plt
import sigpy as sp
import traceback
import copy
import shutil
from pathlib import Path

from dosma import preferences
from dosma.scan_sequences import QDess, CubeQuant, Cones
from sigpy.plot import ImagePlot
from dosma import ImageDataFormat
from tkinter import font

from utils.signed_maurer_distance_map_image_filter import signed_maurer_distance_map_image_filter

2025-01-13 15:04:53.644036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-13 15:04:55.442762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
import dosma as dm

# print(mskt.__version__)

In [2]:
os.environ['DIR']= '/dataNAS/people/anoopai/DODGR'
os.environ['DATA_DICOM']= '/dataNAS/people/anoopai/DODGR/DODGR_data_dicom'
os.environ['DATA']= '/dataNAS/people/anoopai/DODGR/DODGR_data'
os.environ['FILES']= '/dataNAS/people/anoopai/DODGR/code/files'
os.environ['REG']= '/dataNAS/people/anoopai/DODGR/reg'
os.environ['RESULTS'] = '/dataNAS/people/anoopai/DODGR/results'

# Change working directory to the directory containing data
dir_path = os.environ['DIR']
dicom_data_path = os.environ['DATA_DICOM']
data_path = os.environ['DATA']
files_path = os.environ['FILES']
reg_path = os.environ['REG']
results_path = os.environ['RESULTS']

# Set elastix environment variables
elastix_folder = '/dataNAS/people/anoopai/elastix'
os.environ['PATH'] = f"{elastix_folder}/bin:{os.environ['PATH']}"
if 'LD_LIBRARY_PATH' in os.environ:
    os.environ['LD_LIBRARY_PATH'] = f"{elastix_folder}/lib:{os.environ['LD_LIBRARY_PATH']}"
else:
    os.environ['LD_LIBRARY_PATH'] = f"{elastix_folder}/lib"

#!elastix --version
try:
    version_output = subprocess.check_output(['elastix', '--version'], stderr=subprocess.STDOUT, text=True)
    print("Elastix version information:")
    print(version_output)
except subprocess.CalledProcessError as e:
    print("Error running elastix --version:")
    print(e.output)

Elastix version information:
elastix version: 4.900



# Flip all the CL knees as IDX knee

In [3]:
# elastix_file_path = os.path.join(files_path, 'elastic_parameters/elastix_registration_parameters_SDF_mask.txt')

# # idensify baseline and follow-up scans for ACLR and contralateral legs
# for sub_dir in os.listdir(data_path):
#     sub_dir_path = os.path.join(data_path, sub_dir)
    
#     bsln_dirs= [dir for dir in os.listdir(sub_dir_path) if 'bsln' in dir]
#     fup_dirs= [dir for dir in os.listdir(sub_dir_path) if '6mo' in dir]
    
#     visit_dirs = [bsln_dirs, fup_dirs]
    
#     for visit_dir in visit_dirs:
#         print(visit_dir)
#         for knee_dir in visit_dir:
#             print(knee_dir)
#             if 'IDX' in knee_dir:
#                 IDX_dir_path = os.path.join(sub_dir_path, knee_dir)
#             elif 'CL' in knee_dir:
#                 CL_dir_path = os.path.join(sub_dir_path, knee_dir)   
                
#         IDX_qdess_path = os.path.join(IDX_dir_path, 'qdess')
#         IDX_seg_path = os.path.join(IDX_dir_path, 'segmentation_fc.nii')
#         IDX_qdess_reg_path = os.path.join(IDX_dir_path, 'qdess_regToCl')
        
#         CL_qdess_path = os.path.join(CL_dir_path, 'qdess')
#         CL_seg_path = os.path.join(CL_dir_path, 'segmentation_fc.nii')
        
#         knee= knee_dir.split('_')[1]
#         sub = knee_dir.split('_')[0]
#         visit= knee_dir.split('_')[2]
        
#         if not os.path.exists(IDX_qdess_reg_path):
        
#             print(f'Registering IDX to CL for {visit} leg of {sub}')
            
#             sgd_based_bilateral_qdess_registration(
#             fixed_img_path = CL_qdess_path,
#             moving_img_path = IDX_qdess_path,
#             moving_img_save_path = IDX_qdess_reg_path,
#             fixed_mask_path = CL_seg_path,
#             moving_mask_path = IDX_seg_path,
#             elastix_file_path = elastix_file_path,
#             reg_path = reg_path,  
#             reg_check = True  # if True, it will save the a jpg pciture of a random slice (n) of the segmentation mask of the fixed image overlayed on the registered moving image)
#             ) 

In [32]:
elastix_file_path = os.path.join(files_path, 'elastic_parameters/elastix_registration_parameters_SDF_mask.txt')

elastix_file_path = os.path.join(files_path, 'elastic_parameters/elastix_registration_parameters_SDF_mask.txt')
moving_img_path= '/dataNAS/people/anoopai/DODGR/DODGR_data/DODGR023_anon/DODGR023_IDX_6mo_anon/qdess'
moving_mask_path = '/dataNAS/people/anoopai/DODGR/DODGR_data/DODGR023_anon/DODGR023_IDX_6mo_anon/segmentation_fc.nii'
fixed_img_path = '/dataNAS/people/anoopai/DODGR/DODGR_data/DODGR023_anon/DODGR023_CL_6mo_anon/qdess'
fixed_mask_path = '/dataNAS/people/anoopai/DODGR/DODGR_data/DODGR023_anon/DODGR023_CL_6mo_anon/segmentation_fc.nii'
moving_img_save_path = '/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reg2CL'
moving_img_reornt_path = '/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reornt'
seg_reg_SDF_save_path = '/dataNAS/people/anoopai/DODGR/DODGR_data/seg_reg_SDF.nii'
moving_img_save_echo1_path='/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reg2CL_echo1.nii'

In [4]:
output_path= os.path.join(reg_path, f'registration_files')

shutil.rmtree(output_path, ignore_errors=True)

if not os.path.exists(output_path):
    os.makedirs(output_path)

# Delete anything in the output directory.
# This is to make sure there aren't any lingering files that cause the run to succeed sometimes.
shutil.rmtree(moving_img_save_path, ignore_errors=True)

# Load qdess scan
qdess_fixed = QDess.load(fixed_img_path)
qdess_moving = QDess.load(moving_img_path)

# Get echos of the qdess
qdess_fixed_echo1 = qdess_fixed.volumes[0]
qdess_fixed_echo2 = qdess_fixed.volumes[1]  
qdess_moving_echo1 = qdess_moving.volumes[0]
qdess_moving_echo2 = qdess_moving.volumes[1]

# reformat the medical volumes (so that both legs are either right or left side)
qdess_fixed_ornt = qdess_fixed_echo1.orientation

# Flip the orientation of the moving qdess scans
flipped_ornt= tuple('LR' if x == 'RL' else 'RL' if x == 'LR' else x for x in qdess_fixed_ornt)

# Reformat the moving qdess scans
qdess_moving_echo1_reornt= qdess_moving_echo1.reformat(flipped_ornt)
qdess_moving_echo2_reornt= qdess_moving_echo2.reformat(flipped_ornt)
qdess_moving_echo1_reornt_mv= dm.MedicalVolume(qdess_moving_echo1_reornt.A, qdess_fixed_echo1.affine)
qdess_moving_echo2_reornt_mv= dm.MedicalVolume(qdess_moving_echo2_reornt.A, qdess_fixed_echo2.affine)

# load the segmentation for femoral cartilage
reader = dm.NiftiReader()
seg_fixed = reader.load(fixed_mask_path)
seg_moving = reader.load(moving_mask_path)
seg_moving_reornt = seg_moving.reformat(flipped_ornt)
seg_moving_reornt_mv= dm.MedicalVolume(seg_moving_reornt.A, seg_fixed.affine)  

# Combine to form a Qdess Scan
qdess_moving_reornt = QDess([qdess_moving_echo1_reornt_mv, qdess_moving_echo1_reornt_mv])
qdess_moving_reornt.save(moving_img_reornt_path, save_custom=True, image_data_format=ImageDataFormat.nifti)

seg_fixed_binary= np.where(seg_fixed.A > 0, 1, seg_fixed.A)
seg_moving_binary= np.where(seg_moving_reornt_mv.A > 0, 1, seg_moving_reornt_mv.A)

seg_signed_fixed= signed_maurer_distance_map_image_filter(seg_fixed_binary)
seg_signed_fixed= seg_signed_fixed.astype(np.float64)
seg_signed_threshold_0= np.where(seg_signed_fixed > 5, 1, seg_signed_fixed)
seg_signed_mv_0 = dm.MedicalVolume(seg_signed_threshold_0, seg_fixed.affine)

seg_signed_moving= signed_maurer_distance_map_image_filter(seg_moving_binary)
seg_signed_moving= seg_signed_moving.astype(np.float64)
seg_signed_threshold_1= np.where(seg_signed_moving > 5, 1, seg_signed_moving)
seg_signed_mv_1 = dm.MedicalVolume(seg_signed_threshold_1, seg_moving_reornt_mv.affine)

# Registration
# print("Starting Registration")

output= dm.register(
    target=seg_signed_mv_0, 
    moving=seg_signed_mv_1, 
    parameters=elastix_file_path, 
    output_path=output_path, 
    return_volumes=True, 
    num_workers= 50,
    num_threads= 50,
    show_pbar=True)

print("Registration completed")

# gather the registration parameters
registration_parameters = output["outputs"][0]
registered_mask_1 = output['volume'][0]
# registered_mask_1.save_volume(seg_reg_SDF_save_path)

# Warp the Qdess scan using the registration parametrs obtained in the mask registration
reg_echo1 = dm.apply_warp(qdess_moving_echo1_reornt_mv, out_registration=registration_parameters)
reg_echo2 = dm.apply_warp(qdess_moving_echo2_reornt_mv, out_registration=registration_parameters)

# Recontrust the qdess scans from the registered qdess echos
qdess_moving_reg = QDess([reg_echo1, reg_echo2]) # make a qdess \  

# save qdess scans
print(f'Saving registered qdess files for {os.path.basename(moving_img_path)}')
qdess_moving_reg.save(moving_img_save_path, save_custom=True, image_data_format=ImageDataFormat.nifti)


nib_reg_echo1 = nib.Nifti1Image(reg_echo1.A, reg_echo1.affine)
moving_img_save_echo1_path= os.path.join(moving_img_save_path, 'volumes/echo1.nii.gz')
nib_reg_echo2 = nib.Nifti1Image(reg_echo2.A, reg_echo2.affine)
moving_img_save_echo2_path= os.path.join(moving_img_save_path, 'volumes/echo2.nii.gz')


nib.save(nib_reg_echo1, moving_img_save_echo1_path)

print("Done")

100%|██████████| 1/1 [02:20<00:00, 140.95s/it]


Registration completed
Saving registered qdess files for qdess


'/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reg2CL/qdess.data'

In [52]:
nib_reg_echo1 = nib.Nifti1Image(reg_echo1.A, reg_echo1.affine)
moving_img_save_echo1_path= os.path.join(moving_img_save_path, 'volumes/echo1.nii.gz')
nib
nib_reg_echo2 = nib.Nifti1Image(reg_echo2.A, reg_echo2.affine)
moving_img_save_echo2_path= os.path.join(moving_img_save_path, 'volumes/echo2.nii.gz')

In [56]:
reg_echo1_mv = dm.MedicalVolume(reg_echo1.A, reg_echo1.affine)
reg_echo2_mv = dm.MedicalVolume(reg_echo2.A, reg_echo2.affine)

qdess_moving_reg = QDess([reg_echo1_mv, reg_echo2_mv]) # make a qdess \  

moving_img_save_path_2 = '/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reg2CL2'

# save qdess scans
print(f'Saving registered qdess files for {os.path.basename(moving_img_path)}')
qdess_moving_reg.save(moving_img_save_path_2, save_custom=True, image_data_format=ImageDataFormat.nifti)


Saving registered qdess files for qdess


'/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reg2CL2/qdess.data'

In [ ]:
reg_echo1.save_volume(moving_img_save_echo1_path)

In [29]:
qdess_sitk= qdess_fixed_echo1.to_sitk()

data= sitk.GetArrayFromImage(qdess_sitk)
print(qdess_sitk.GetDirection())
print(qdess_sitk.GetOrigin())
print(qdess_sitk.GetSpacing())

(0.030301224951778512, -0.028999403198423205, -0.9991230290976285, 0.9995404072377104, -0.0, 0.03026635452127333, -0.0009000363847062924, -0.9995794288670287, 0.028933034938838382)
(-54.3395, -93.0797, 72.1025)
(0.31248736693214896, 0.3125064311838238, 1.500015469920227)


In [28]:
qdess_sitk= qdess_moving_echo1.to_sitk()

data= sitk.GetArrayFromImage(qdess_sitk)
print(qdess_sitk.GetDirection())
print(qdess_sitk.GetOrigin())
print(qdess_sitk.GetSpacing())

(-0.20689506974923114, 0.09179794007733674, -0.9740477789810817, 0.978176690327201, -0.0, -0.20779597064431124, -0.0190995448632688, -0.9957776550001299, -0.0897315933549063)
(153.98, -84.3011, 84.6553)
(0.31250744678627257, 0.3125070124213215, 1.5000290863846608)


In [26]:
qdess_sitk= qdess_moving_echo1_reornt.to_sitk()

data= sitk.GetArrayFromImage(qdess_sitk)
print(qdess_sitk.GetDirection())
print(qdess_sitk.GetOrigin())
print(qdess_sitk.GetSpacing())

(-0.20689506974923114, 0.09179794007733674, -0.9740477789810817, 0.978176690327201, -0.0, -0.20779597064431124, -0.0190995448632688, -0.9957776550001299, -0.0897315933549063)
(153.98, -84.3011, 84.6553)
(0.31250744678627257, 0.3125070124213215, 1.5000290863846608)


In [31]:
qdess_sitk= qdess_moving_echo1_reornt_mv.to_sitk()

data= sitk.GetArrayFromImage(qdess_sitk)
print(qdess_sitk.GetDirection())
print(qdess_sitk.GetOrigin())
print(qdess_sitk.GetSpacing())

(0.030301224951778512, -0.028999403198423205, -0.9991230290976285, 0.9995404072377104, -0.0, 0.03026635452127333, -0.0009000363847062924, -0.9995794288670287, 0.028933034938838382)
(-54.3395, -93.0797, 72.1025)
(0.31248736693214896, 0.3125064311838238, 1.500015469920227)


In [49]:
qdess_sitk= reg_echo1.to_sitk()

data= sitk.GetArrayFromImage(qdess_sitk)
affine_matrix= qdess_sitk.GetDirection()
print(np.array(affine_matrix).reshape(3, 3))
print(qdess_sitk.GetOrigin())
print(qdess_sitk.GetSpacing())

[[ 3.04053900e-02 -2.91169709e-02 -9.99119082e-01]
 [ 9.99537188e-01 -0.00000000e+00  3.03329007e-02]
 [-9.60170209e-04 -9.99576011e-01  2.89995863e-02]]
(-54.3395, -93.0876, 72.0946)
(0.3124446030898918, 0.3125325103089277, 1.500021396514063)


In [35]:
sitk.WriteImage(qdess_sitk, moving_img_save_echo1_path)

NiftiImageIO (0x6ac71c0): Non-orthogonal direction matrix coerced to orthogonal



In [21]:
qdess_fixed_echo1, qdess_moving_echo1_reornt, qdess_moving_echo1_reornt_mv, reg_echo1

(MedicalVolume(
   shape=(80, 512, 512),
   ornt=('LR', 'SI', 'AP')),
   spacing=(1.500015469920227, 0.3125064311838238, 0.31248736693214896),
   origin=(54.3395, 93.0797, 72.1025),
   device=Device(type='cpu')
 ),
 MedicalVolume(
   shape=(80, 512, 512),
   ornt=('LR', 'SI', 'AP')),
   spacing=(1.5000290863846608, 0.3125070124213215, 0.31250744678627257),
   origin=(-153.98, 84.3011, 84.6553),
   device=Device(type='cpu')
 ),
 MedicalVolume(
   shape=(80, 512, 512),
   ornt=('LR', 'SI', 'AP')),
   spacing=(1.500015469920227, 0.3125064311838238, 0.31248736693214896),
   origin=(54.3395, 93.0797, 72.1025),
   device=Device(type='cpu')
 ),
 MedicalVolume(
   shape=(80, 512, 512),
   ornt=('LR', 'SI', 'AP')),
   spacing=(1.500021396514063, 0.3125325103089277, 0.3124446030898918),
   origin=(54.3395, 93.0876, 72.0946),
   device=Device(type='cpu')
 ))

In [22]:
qdess_fixed_echo1.affine, qdess_moving_echo1_reornt.affine, qdess_moving_echo1_reornt_mv.affine, reg_echo1.affine

(array([[ 1.4987000e+00,  9.0625000e-03, -9.4687500e-03,  5.4339500e+01],
        [-4.5400000e-02,  0.0000000e+00, -3.1234375e-01,  9.3079700e+01],
        [ 4.3400000e-02, -3.1237500e-01, -2.8125000e-04,  7.2102500e+01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]]),
 array([[ 1.461100e+00, -2.868750e-02,  6.465625e-02, -1.539800e+02],
        [ 3.117000e-01,  0.000000e+00, -3.056875e-01,  8.430110e+01],
        [-1.346000e-01, -3.111875e-01, -5.968750e-03,  8.465530e+01],
        [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  1.000000e+00]]),
 array([[ 1.4987000e+00,  9.0625000e-03, -9.4687500e-03,  5.4339500e+01],
        [-4.5400000e-02,  0.0000000e+00, -3.1234375e-01,  9.3079700e+01],
        [ 4.3400000e-02, -3.1237500e-01, -2.8125000e-04,  7.2102500e+01],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]]),
 array([[ 1.49870e+00,  9.10000e-03, -9.50000e-03,  5.43395e+01],
        [-4.55000e-02,  0.00000e+00, -3.12300e-01,  9.30

In [ ]:
seg_moving_reornt_mv.affine,  qdess_moving_echo2_reornt_mv.affine

In [ ]:
qdess_moving_reornt.save(moving_img_reornt_path, save_custom=True, image_data_format=ImageDataFormat.nifti)


In [51]:
# affine_matrix= qdess_moving_reornt.volumes[0].affine

# Extract the rotation matrix (the first 3x3 part of the affine matrix)
rotation_matrix = np.array(affine_matrix).reshape(3, 3)

# Check orthogonality: dot product of different vectors should be zero
dot_product_01 = np.dot(rotation_matrix[:, 0], rotation_matrix[:, 1])
dot_product_02 = np.dot(rotation_matrix[:, 0], rotation_matrix[:, 2])
dot_product_12 = np.dot(rotation_matrix[:, 1], rotation_matrix[:, 2])

# Check unit length: the magnitude of each vector should be 1
length_0 = np.linalg.norm(rotation_matrix[:, 0])
length_1 = np.linalg.norm(rotation_matrix[:, 1])
length_2 = np.linalg.norm(rotation_matrix[:, 2])

# Print results
print("Dot product of column 0 and 1 (should be 0):", dot_product_01)
print("Dot product of column 0 and 2 (should be 0):", dot_product_02)
print("Dot product of column 1 and 2 (should be 0):", dot_product_12)
print("Length of column 0 (should be 1):", length_0)
print("Length of column 1 (should be 1):", length_1)
print("Length of column 2 (should be 1):", length_2)


Dot product of column 0 and 1 (should be 0): 7.445025389546682e-05
Dot product of column 0 and 2 (should be 0): -8.758761084072306e-05
Dot product of column 1 and 2 (should be 0): 0.00010403035999833471
Length of column 0 (should be 1): 1.0
Length of column 1 (should be 1): 1.0
Length of column 2 (should be 1): 1.0


In [46]:
rotation_matrix.shape

(4, 4)

In [50]:
affine_matrix

(0.03040538996689536,
 -0.029116970874501862,
 -0.9991190815563472,
 0.9995371880696234,
 -0.0,
 0.03033290065444305,
 -0.0009601702094809061,
 -0.9995760111202616,
 0.028999586339962035)

In [ ]:
# gather the registration parameters
registration_parameters = output["outputs"][0]
registered_mask_1 = output['volume'][0]
# registered_mask_1.save_volume(seg_reg_SDF_save_path)

# Warp the Qdess scan using the registration parametrs obtained in the mask registration
reg_echo1 = dm.apply_warp(qdess_moving_echo1_reornt_mv, out_registration=registration_parameters)
reg_echo2 = dm.apply_warp(qdess_moving_echo2_reornt_mv, out_registration=registration_parameters)

# Recontrust the qdess scans from the registered qdess echos
qdess_moving_reg = QDess([reg_echo1, reg_echo2]) # make a qdess \  

# save qdess scans
print(f'Saving registered qdess files for {os.path.basename(moving_img_path)}')
qdess_moving_reg.save(moving_img_save_path, save_custom=True, image_data_format=ImageDataFormat.nifti)

reg_check_path = os.path.join(reg_path, f'registration_check')

if not os.path.exists(reg_check_path):
    os.makedirs(reg_check_path)             

# Plot and save registrations for checking
n= 30
_, axes = plt.subplots(nrows=1, ncols=2, figsize=(20,20), constrained_layout=False)
axes[0].imshow(qdess_fixed.volumes[0].A[:,:,n], cmap='gray', alpha=1)
axes[0].imshow(qdess_moving_reg.volumes[0].A[:,:,n], cmap="Blues", alpha= 0.7)
axes[1].imshow(qdess_moving_reg.volumes[0].A[:,:,n], cmap='gray', alpha=1)
axes[1].imshow(seg_fixed.A[:,:,n], cmap="Blues", alpha=0.7)
axes[0].set_title("Fixed vs Moving Qdess Scan", fontsize=12)
axes[1].set_title("Fixed Mask on Moving Qdess Scan", fontsize=12)

plt_save_path= os.path.join(reg_check_path, f'{moving_img_path.split(os.sep)[-2]}_slice{n}.jpg')
plt.savefig(plt_save_path)


In [ ]:
data= QDess.load(moving_img_save_path)  # Load the registered QDess scan
affine= data.volumes[1].affine

U, _, Vt = np.linalg.svd(affine[:3, :3], full_matrices=False)
orthonormal_affine = np.dot(U, Vt)
orthonormal_affine
orthonormal_affine

In [ ]:
data= QDess.load(fixed_img_path)  # Load the registered QDess scan
affine= data.volumes[1].affine

U, _, Vt = np.linalg.svd(affine[:3, :3], full_matrices=False)
orthonormal_affine = np.dot(U, Vt)
orthonormal_affine

In [ ]:
qdess_moving_reg.save(moving_img_save_path, save_custom=True, image_data_format=ImageDataFormat.nifti)


In [ ]:
fig, ax= plt.subplots(1, 1, figsize=(10, 5))

# plt.imshow(qdess_fixed_echo1.A[:, :, 50], cmap='gray')
plt.imshow(reg_echo1.A[:, :, 40]) 

In [ ]:
np.unique(registered_mask_1.A)

In [ ]:
plt.imshow(registered_mask_1.A[:, :, 40])

In [ ]:
qdess_moving_echo1_reornt.affine

In [ ]:
qdess_moving_reg.save(moving_img_save_path, save_custom=True)

In [ ]:
qdess_fixed_echo1.affine, qdess_moving_echo1.affine, qdess_moving_echo1_reornt.affine, reg_echo1.affine

In [ ]:
data_path= '/dataNAS/people/anoopai/DODGR/reg/registration_files/moving-0/TransformParameters.0.txt'

# load data_path

with open(data_path, 'rb') as f:
    qdess_data = pickle.load(f)
    
qdess_data

In [ ]:
data_path= '/dataNAS/people/anoopai/DODGR/DODGR_data/qdess_reornt/qdess.data'

# load data_path

with open(data_path, 'rb') as f:
    qdess_data = pickle.load(f)
    
qdess_data

In [ ]:
qdess_fixed_echo1.is_identical(qdess_moving_echo1_reornt_mv)

qdess_moving_echo1_reornt.reformat_as(qdess_fixed_echo1)

qdess_fixed_echo1.is_identical(qdess_moving_echo1_reornt_mv)
# qdess_fixed_echo1.affine, qdess_moving_echo1_reornt_mv.affine

qdess_fixed_echo1.is_same_dimensions(qdess_moving_echo1_reornt_mv)

In [ ]:
qdess_fixed_echo1, qdess_moving_echo1, qdess_moving_echo1_reornt, qdess_moving_echo1_reornt_mv

In [ ]:
qdess = QDess.load(moving_img_save_path)
qdess.volumes[0]

In [14]:
qdess_moving_echo1_flip= np.flip(qdess_moving_echo1.A, axis=1) 

In [ ]:
qdess_moving_echo1.affine, qdess_fixed_echo1.affine

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
plt.imshow(qdess_moving_echo1_reornt.A[:, :, 50], cmap='gray')
plt.imshow(qdess_fixed_echo1.A[:, :, 50], cmap='Blues', alpha=0.5)

In [45]:
import SimpleITK as sitk

sitk.__version__
g= sitk.ReadImage('/dataNAS/people/anoopai/DODGR/DODGR_data/DODGR007_anon/DODGR007_IDX_bsln_anon/segmentation_regToCl_fc_subregions.nii')

In [ ]:
qdess_moving_echo1_reornt, qdess_moving_echo1_reornt.affine

In [ ]:
qdess_fixed_echo1, qdess_fixed_echo1.affine

In [ ]:
pl, ax= plt.subplots(1, 1, figsize=(10, 10))
n=20
plt.imshow(qdess_fixed_echo1.A[:,:,n], cmap='gray')
# plt.imshow(qdess_moving_echo1_reornt.A[:,:,n], cmap='Reds', alpha=0.5)
plt.imshow(qdess.volumes[0].A[:,:,n], cmap='Blues', alpha=0.4)


In [10]:
# from utils.signed_maurer_distance_map_image_filter import signed_maurer_distance_map_image_filter

# seg_signed_fixed= signed_maurer_distance_map_image_filter(seg_fixed.A)
# seg_signed_fixed= seg_signed_fixed.astype(np.float64)
# seg_signed_threshold_0= np.where(seg_signed_fixed > 5, 1, seg_signed_fixed)
# seg_signed_mv_0 = dm.MedicalVolume(seg_signed_threshold_0, qdess_fixed_echo1.affine)

# seg_signed_moving= signed_maurer_distance_map_image_filter(seg_moving_reornt.A)
# seg_signed_moving= seg_signed_moving.astype(np.float64)
# seg_signed_threshold_1= np.where(seg_signed_moving > 5, 1, seg_signed_moving)
# seg_signed_mv_1 = dm.MedicalVolume(seg_signed_threshold_1, qdess_moving_echo1.affine)

# # Registration
# print("Starting Registration")

# output= dm.register(
#     target=seg_signed_mv_0, 
#     moving=seg_signed_mv_1, 
#     parameters=elastix_file_path, 
#     output_path=output_path, 
#     return_volumes=True, 
#     num_workers= 50,
#     num_threads= 50,
#     show_pbar=True)

# print("Registration completed")

# # gather the registration parameters
# registration_parameters = output["outputs"][0]
# registered_mask_1 = output['volume'][0]
# # registered_mask_1.save_volume(seg_reg_SDF_save_path)

# # Warp the Qdess scan using the registration parametrs obtained in the mask registration
# reg_echo1 = dm.apply_warp(qdess_moving_echo1_reornt, out_registration=registration_parameters)
# reg_echo2 = dm.apply_warp(qdess_moving_echo2_reornt, out_registration=registration_parameters)

# # Recontrust the qdess scans from the registered qdess echos
# qdess_moving_reg = QDess([reg_echo1, reg_echo2]) # make a qdess \

# # save qdess scans
# print(f'Saving registered qdess files for {os.path.basename(moving_img_path)}')
# qdess_moving_reg.save(moving_img_save_path, save_custom=True, image_data_format=ImageDataFormat.nifti)